In [2]:
import pandas as pd

class Process:
    def __init__(self, name, arrival, burst, priority):
        self.name = name
        self.arrival = arrival
        self.burst = burst
        self.priority = priority
        self.remaining = burst
        self.turnaround = 0
        self.waiting = 0
        self.start = 0
        self.finish = 0
        self.q = 0

def get_process_input():
    name = input("Enter process name: ")
    arrival = int(input("Enter arrival time: "))
    burst = int(input("Enter burst time: "))
    priority = int(input("Enter priority (0 for High, 1 for Medium, 2 for Low): "))
    return Process(name, arrival, burst, priority)

def generate_output_dataframe(processes):
    total_waiting = sum(p.waiting for p in processes)
    average_waiting = total_waiting / len(processes)
    
    data = {
        "Process": [p.name for p in processes],
        "Turnaround Time": [p.turnaround for p in processes],
        "Waiting Time": [p.waiting for p in processes],
    }
    df = pd.DataFrame(data)
    return df, average_waiting

def MLQ(processes):
    processes.sort(key=lambda p: p.arrival)
    high_queue = []  # Queue 0, Round Robin with q = 3
    medium_queue = []  # Queue 1, Shortest Remaining First with q = 2
    low_queue = []  # Queue 2, Shortest Job Next
    time = 0
    current = None

    while True:
        # Check if any process has arrived
        for p in processes:
            if p.arrival == time:
                print("At t = {}, {} arrived the queue {}".format(time, p.name, p.priority))
                if p.priority == 0:
                    high_queue.append(p)
                    p.q = 3
                elif p.priority == 1:
                    medium_queue.append(p)
                    p.q = 2
                else:
                    low_queue.append(p)

        # Check if the current process is finished
        if current and current.remaining == 0:
            print("At t = {}, {} left the kernel and finished".format(time, current.name))
            current.finish = time
            current.turnaround = current.finish - current.arrival
            current.waiting = current.turnaround - current.burst
            current = None

        # Check if the current process is preempted
        if current and current.q == 0:
            print("At t = {}, {} left the kernel and entered the queue {}".format(time, current.name, current.priority))
            if current.priority == 0:
                current.q = 3
                high_queue.append(current)
            else:
                current.q = 2
                medium_queue.append(current)
            current = None

        # Check if the current process is None
        if not current:
            if high_queue:
                current = high_queue.pop(0)
            elif medium_queue:
                medium_queue.sort(key=lambda p: p.remaining)
                current = medium_queue.pop(0)
            elif low_queue:
                low_queue.sort(key=lambda p: p.burst)
                current = low_queue.pop(0)
            else:
                break

            if current.start == -1:
                current.start = time
            print("At t = {}, {} entered the kernel".format(time, current.name))

        current.remaining -= 1
        current.q -= 1
        time += 1

    output_df, average_waiting = generate_output_dataframe(processes)
    print("\nProcess details after scheduling:")
    print(output_df)
    print("\nAverage Waiting Time: {:.4f}".format(average_waiting))

# Get user input for the number of processes
num_processes = int(input("Enter the number of processes: "))
# Assuming you have a function `get_process_input` to get details for each process
processes = [get_process_input() for _ in range(num_processes)]

# Call the MLQ scheduling function with user-input processes
MLQ(processes)


Enter the number of processes: 6
Enter process name: p0
Enter arrival time: 0
Enter burst time: 5
Enter priority (0 for High, 1 for Medium, 2 for Low): 1
Enter process name: p1
Enter arrival time: 1
Enter burst time: 8
Enter priority (0 for High, 1 for Medium, 2 for Low): 0
Enter process name: p2
Enter arrival time: 3
Enter burst time: 6
Enter priority (0 for High, 1 for Medium, 2 for Low): 2
Enter process name: p3
Enter arrival time: 5
Enter burst time: 4
Enter priority (0 for High, 1 for Medium, 2 for Low): 2
Enter process name: p4
Enter arrival time: 8
Enter burst time: 2
Enter priority (0 for High, 1 for Medium, 2 for Low): 1
Enter process name: p5
Enter arrival time: 16
Enter burst time: 10
Enter priority (0 for High, 1 for Medium, 2 for Low): 0
At t = 0, p0 arrived the queue 1
At t = 0, p0 entered the kernel
At t = 1, p1 arrived the queue 0
At t = 2, p0 left the kernel and entered the queue 1
At t = 2, p1 entered the kernel
At t = 3, p2 arrived the queue 2
At t = 5, p3 arrived th